### Testing Evo

#### Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch_geometric
from torch_geometric.nn import GCNConv
import IPython
from IPython.display import display, HTML, Image
from torch_geometric.loader import DataLoader

from graphUtils import add_edges
from generator import generate_organism
from cell import Cell
from organism import Organism
from GNCAmodel import GNCA
from visualizer import Visualizer
from evo_trainer import Evo_Trainer

%run graphUtils.py
%run Visualizer.py # --allows for reloading the particular code
#%run CGConv.py

#### Train the network

In [2]:
n = 10
popsize = 10
generations = 200
batch_size = 2
wrap_around = True
name = 'centered13'
#device = torch.device('mps')
device = torch.device('cpu')
trainer = Evo_Trainer(n, device, batch_size, True, popsize)

In [ ]:
trainer.train(generations, name)

(EvaluationActor pid=31448) dynamic edges:  0.3194034999978612
(EvaluationActor pid=31448) update graph:  0.01976483299949905
(EvaluationActor pid=31448) metrics:  0.0017858330029412173
(EvaluationActor pid=31448) remove nodes:  0.00042341699736425653
(EvaluationActor pid=31454) dynamic edges:  0.3684829589983565
(EvaluationActor pid=31454) update graph:  0.0042697910030256025
(EvaluationActor pid=31454) metrics:  0.0010442499988130294
(EvaluationActor pid=31454) remove nodes:  0.00045037500240141526
(EvaluationActor pid=31453) dynamic edges:  0.3299257079997915
(EvaluationActor pid=31453) update graph:  0.004129624998313375
(EvaluationActor pid=31453) metrics:  0.0010217500021099113
(EvaluationActor pid=31453) remove nodes:  0.0004318749997764826
(EvaluationActor pid=31455) dynamic edges:  0.32748241600347683
(EvaluationActor pid=31455) update graph:  0.004406833999382798
(EvaluationActor pid=31455) metrics:  0.0010496249960851856
(EvaluationActor pid=31455) remove nodes:  0.000446791

KeyboardInterrupt: 

(EvaluationActor pid=31448) dynamic edges:  0.13344975000654813
(EvaluationActor pid=31448) update graph:  0.0018916669941972941
(EvaluationActor pid=31448) metrics:  0.0004850830009672791
(EvaluationActor pid=31448) remove nodes:  0.0002904170032707043
(EvaluationActor pid=31454) dynamic edges:  0.1285005000027013
(EvaluationActor pid=31454) update graph:  0.002517541994166095
(EvaluationActor pid=31454) metrics:  0.0005630000014207326
(EvaluationActor pid=31454) remove nodes:  0.00031033300183480605
(EvaluationActor pid=31452) dynamic edges:  0.09729595800308743
(EvaluationActor pid=31452) update graph:  0.0016780419973656535
(EvaluationActor pid=31452) metrics:  0.0004927920017507859
(EvaluationActor pid=31452) remove nodes:  0.0002971659996546805
(EvaluationActor pid=31450) dynamic edges:  0.11872370899800444
(EvaluationActor pid=31450) update graph:  0.0016907909957808442
(EvaluationActor pid=31450) metrics:  0.0005108750046929345
(EvaluationActor pid=31450) remove nodes:  0.00031

In [ ]:
network = trainer.get_trained_network()

In [ ]:
#trainer.logger_df.from_csv('../logger/' + name + '.csv')
network.load_state_dict(torch.load('../models/' + name + '.pth'))

In [ ]:
#network = trainer.problem.parameterize_net(trainer.searcher.status['best'][0])
network = trainer.problem.parameterize_net(trainer.searcher.status['center'][0])
#network = trainer.problem.parameterize_net(trainer.searcher.status['obj0_pop_best'])
#network = trainer.problem.parameterize_net(trainer.searcher.status['obj4_pop_best'])

In [ ]:
#trainer.searcher.status

#### Visualize trained network

In [ ]:
graphs = [generate_organism(n, device).toGraph() for _ in range(1)]
loader = DataLoader(graphs, batch_size=1)
graph = next(iter(loader))
visualizer = Visualizer(wrap_around)
network.batch_size = 1
network(graph, 0)

In [ ]:
anim = visualizer.animate_organism(graph, network, food=30, frames=200)
visualizer.save_animation_to_gif(anim, name, 10)

In [ ]:
Image('../animation/' + name + '.gif')

In [ ]:
Image('../animation/GATConv1.gif')

In [ ]:
anim.frame_seq = anim.new_saved_frame_seq() 
IPython.display.HTML(anim.to_jshtml())

In [ ]:
print(network)

In [ ]:
print('GConvGRU: ', network.gConvGRU)
for param in network.gConvGRU.parameters():
  print(param.data)


In [ ]:
%%javascript
IPython.notebook.save_notebook()

#### Visualization of loss:

In [ ]:
import matplotlib.pyplot as plt
losses = trainer.logger_df
#print(losses)
#plt.plot(losses.stepsize)

plt.plot(losses.median_eval)
plt.plot(losses.pop_best_eval)
plt.plot(losses.mean_eval)
#plt.legend(('median_eval', 'pop_best_eval', 'mean eval'))
plt.show()

In [ ]:
trainer.logger_df
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
fig.suptitle('')
plt.show()